In [1]:
!pip install pyyaml h5py


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
def create_model():
    model = tf.keras.models.Sequential(
        [
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
        ]
    )
    model.compile(optimizer='adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseTopKCategoricalAccuracy()])
    
    return model

In [5]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images,test_labels),
    callbacks=[cp_callback]

)

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2329 - sparse_top_k_categorical_accuracy: 0.9007
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 26ms/step - loss: 1.1706 - sparse_top_k_categorical_accuracy: 0.9070 - val_loss: 0.6874 - val_sparse_top_k_categorical_accuracy: 0.9870
Epoch 2/10
17/32 [==============>...............] - ETA: 0s - loss: 0.4605 - sparse_top_k_categorical_accuracy: 0.9945
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.4221 - sparse_top_k_categorical_accuracy: 0.9950 - val_loss: 0.5212 - val_sparse_top_k_categorical_accuracy: 0.9920
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2596 - sparse_top_k_categorical_accuracy: 0.9976
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 0.2818 - sparse_top_k_categorical_accuracy: 0.9950 - val_loss: 0.4822 - val_sparse_top_k_

In [7]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [8]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'Model niewytrenowany, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 2.3308 - sparse_top_k_categorical_accuracy: 0.5430 - 191ms/epoch - 6ms/step
Model niewytrenowany, accuracy: 54.30%


In [10]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'Model odtworzony, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4332 - sparse_top_k_categorical_accuracy: 0.9940 - 77ms/epoch - 2ms/step
Model niewytrenowany, accuracy: 99.40%


In [11]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size=32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True,
    verbose=1,
    save_freq=5*batch_size
)

In [12]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    batch_size=batch_size,
    validation_data=(test_images,test_labels),
    callbacks=[cp_callback],
    verbose=0

)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [13]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0050.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001']

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
model = create_model()
model.load_weights(latest)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'Model odtworzony - latest, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4895 - sparse_top_k_categorical_accuracy: 0.9940 - 184ms/epoch - 6ms/step
Model odtworzony - latest, accuracy: 99.40%


In [16]:
model.save_weights('./checkpoints/moj_check')
model = create_model()
model.load_weights('./checkpoints/moj_check')
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'Model odtworzony - latest, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4895 - sparse_top_k_categorical_accuracy: 0.9940 - 189ms/epoch - 6ms/step
Model odtworzony - latest, accuracy: 99.40%


In [17]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.2189 - sparse_top_k_categorical_accuracy: 0.9070
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4366 - sparse_top_k_categorical_accuracy: 0.9940
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2915 - sparse_top_k_categorical_accuracy: 0.9970
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2099 - sparse_top_k_categorical_accuracy: 0.9990
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1601 - sparse_top_k_categorical_accuracy: 0.9990


In [18]:
!mkdir -p saved_model

In [19]:
model.save('saved_model/moj_model')

INFO:tensorflow:Assets written to: saved_model/moj_model/assets


In [20]:
new_model = tf.keras.models.load_model('saved_model/moj_model')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'Model odtworzony - latest, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4672 - sparse_top_k_categorical_accuracy: 0.9900 - 191ms/epoch - 6ms/step
Model odtworzony - latest, accuracy: 99.00%


In [22]:
print(new_model.predict(test_images).shape)

32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [23]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)

model.save('mmodel.h5')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1815 - sparse_top_k_categorical_accuracy: 0.9200
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4341 - sparse_top_k_categorical_accuracy: 0.9930
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2911 - sparse_top_k_categorical_accuracy: 0.9970
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2284 - sparse_top_k_categorical_accuracy: 0.9970
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1627 - sparse_top_k_categorical_accuracy: 0.9990


In [24]:
h5_model = tf.keras.models.load_model('mmodel.h5')
h5_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss,acc = h5_model.evaluate(test_images,test_labels,verbose=2)
print(f'Model odtworzony - latest, accuracy: {100*acc:.2f}%')

32/32 - 0s - loss: 0.4415 - sparse_top_k_categorical_accuracy: 0.9950 - 182ms/epoch - 6ms/step
Model odtworzony - latest, accuracy: 99.50%
